Load the necessary packages

In [1]:
import pickle
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random



Load data

In [2]:
dict = pickle.load(open('elo_dict.pkl', 'rb'))


In [3]:
data = {"link":[], "Elo_home":[], "Elo_away":[]}
for key in dict.keys():
    data["link"].append(key)
    data["Elo_home"].append(dict[key]["Elo_home"])
    data['Elo_away'].append(dict[key]['Elo_away'])


Data processing

In [4]:
df = pd.DataFrame(data = data)
df.dropna(inplace=True)
df.reset_index(inplace=True)
n_rows = df.shape[0]
random.seed(a = 1)
numbers = random.sample(range(1, n_rows+1), 1000)
df = df.loc[numbers]


In [5]:
for i in df.index:
    url = df.loc[i,'link']
    req = requests.get(url+'/table')
    soup = BeautifulSoup(req.content, 'html.parser')
    home = soup.select('p[itemprop=homeTeam]')[0].text[1:-1]
    away = soup.select('p[itemprop=awayTeam]')[0].text[1:-1]
    home_score = soup.select('span[class=r1]')[0].get_text()
    away_score = soup.select('span[class=r2]')[0].get_text()
    home_GF = 0
    home_GA = 0
    away_GF = 0
    away_GA = 0
    for html in soup.select('.row-body.marked'):
        if html.select('span[class=team-name]')[0].get_text() == home:
            home_GF = html.select('td[data-content-tab=tab_cc_total]')[5].get_text()
            home_GA = html.select('td[data-content-tab=tab_cc_total]')[6].get_text()
        elif html.select('span[class=team-name]')[0].get_text() == away:
            away_GF = html.select('td[data-content-tab=tab_cc_total]')[5].get_text()
            away_GA = html.select('td[data-content-tab=tab_cc_total]')[6].get_text()
    df.loc[i, 'home_GF'] = home_GF
    df.loc[i, 'home_GA'] = home_GA
    df.loc[i, 'away_GF'] = away_GF
    df.loc[i, 'away_GA'] = away_GA
    if home_score > away_score:
        df.loc[i, 'result'] = 1
    elif home_score == away_score:
        df.loc[i, 'result'] = 0
    else:
        df.loc[i, 'result'] = -1
df.head()


,index,link,Elo_home,Elo_away,home_GF,home_GA,away_GF,away_GA,result
17612,17918,https://www.besoccer.com/match/tsg-1899-hoffen...,74.0,75.0,49,55,50,65,-1.0
74607,81300,https://www.besoccer.com/match/sporting-lisbon...,87.0,90.0,79,21,88,22,-1.0
111075,117794,https://www.besoccer.com/match/torino-fc/genoa...,77.0,72.0,52,37,39,57,1.0
105155,111874,https://www.besoccer.com/match/empoli-fc/atala...,72.0,72.0,50,58,25,48,1.0
100110,106806,https://www.besoccer.com/match/alaves/tenerife...,57.0,54.0,49,35,45,46,0.0


In [6]:
df.drop(columns='link', inplace=True)
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)
df.head()

,level_0,Elo_home,Elo_away,home_GF,home_GA,away_GF,away_GA,result
0,17612,74.0,75.0,49,55,50,65,-1.0
1,74607,87.0,90.0,79,21,88,22,-1.0
2,111075,77.0,72.0,52,37,39,57,1.0
3,105155,72.0,72.0,50,58,25,48,1.0
4,100110,57.0,54.0,49,35,45,46,0.0


In [7]:
df.to_csv('sampled_matches.csv')

In [8]:
df.loc[[711],]

,level_0,Elo_home,Elo_away,home_GF,home_GA,away_GF,away_GA,result
711,22568,60.0,69.0,59,71,55,51,1.0
